# for Machine learning



In [ ]:
train_size = int(len(df) * .8)

print(int(len(df['text'])))
print(train_size)

# Tokenizing the dataset

In [ ]:
t= df['text']
dial = df['dialect']


k= df['text'][:train_size]
c = df['dialect'][:train_size]



k2 = df['text'][train_size:]
c2 =  df['dialect'][train_size:]

In [ ]:
tokenizer = Tokenizer(num_words=None,lower=False)
tokenizer.fit_on_texts(t)

x_train = tokenizer.texts_to_matrix(k, mode='tfidf')
x_test = tokenizer.texts_to_matrix(k2, mode='tfidf')

In [ ]:
encoder = LabelEncoder()
encoder.fit(dial)
dial2=encoder.fit_transform(dial)

num_classes = int((len(set(dial2))))
print((len(set(dial2))))

y_train = encoder.fit_transform(c)
y_test = encoder.fit_transform(c2)

In [ ]:
y_train= keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


num_labels = int(len(y_train.shape))
vocab_size = len(tokenizer.word_index) + 1

max_words=vocab_size

# Defining the NLP model 

In [ ]:
def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

from keras.metrics import Precision , Recall , Accuracy , TruePositives , TrueNegatives , FalsePositives , FalseNegatives

In [ ]:
model = Sequential()
model.add(Dense(1024, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy','Recall','Precision', f1_metric,'TruePositives','TrueNegatives','FalsePositives','FalseNegatives'])

In [ ]:
batch_size = 100
epochs = 2

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

# For Deep Learning Model 

# embeding vectors for few sample statements

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    " لما يكون جاي على بالك كاسة شاي وتيجي لحالها", 
    "قتلو جون سنو لو بعده للحين"]
)

# Bulid BERT model

In [ ]:
ti = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preptext = bert_preprocess(ti)
outputs = bert_encoder(preptext)
z = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
z = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(z)
model = tf.keras.Model(inputs=[ti], outputs = [z])

In [ ]:
model.summary()

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10)